In [5]:
#Dependecies
import pandas as pd
from path import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

file_path = Path("../Resources/loans_data_encoded.csv")

loans_df = pd.read_csv(file_path)

loans_df

,amount,term,age,bad,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,0,6,0,1,0,0,0,1
1,1000,30,50,0,7,1,0,0,0,1,0
2,1000,30,33,0,8,1,0,0,0,1,0
3,1000,15,27,0,9,0,0,0,1,0,1
4,1000,30,28,0,10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
495,1000,30,28,1,12,0,1,0,0,0,1
496,1000,15,26,1,7,0,1,0,0,0,1
497,800,15,30,1,6,0,0,0,1,0,1
498,1000,30,38,1,3,0,0,0,1,1,0


In [14]:
#Seperating the features and target column
y = loans_df["bad"].values

X = loans_df.copy()
X = X.drop("bad", axis=1)
X.head()

,amount,term,age,month_num,education_Bachelor,education_High School or Below,education_Master or Above,education_college,gender_female,gender_male
0,1000,30,45,6,0,1,0,0,0,1
1,1000,30,50,7,1,0,0,0,1,0
2,1000,30,33,8,1,0,0,0,1,0
3,1000,15,27,9,0,0,0,1,0,1
4,1000,30,28,10,0,0,0,1,1,0


In [15]:
#split the training and 
X_train, X_test, y_train, y_test = train_test_split (X,y,random_state=1)

In [16]:
scaler = StandardScaler()

X_scaler= scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train_scaled, y_train)
   print("Learning rate: ", learning_rate)
    
   print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_train)))
   print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))

Learning rate:  0.05
Accuracy score (training): 0.629
Accuracy score (validation): 0.512
Learning rate:  0.1
Accuracy score (training): 0.656
Accuracy score (validation): 0.520
Learning rate:  0.25
Accuracy score (training): 0.723
Accuracy score (validation): 0.536
Learning rate:  0.5
Accuracy score (training): 0.755
Accuracy score (validation): 0.560
Learning rate:  0.75
Accuracy score (training): 0.781
Accuracy score (validation): 0.520
Learning rate:  1
Accuracy score (training): 0.805
Accuracy score (validation): 0.512


# Result

Of the learning rates used, 0.5 yields the best accuracy score for the testing set and a high accuracy score for the training set. This is the value we'll implement in the final model. 

The GradientBoostingClassifier includes the following:

-An array called learning_rates is manually created and contains a range of values.

-For each learning rate value, a GradientBoostingClassifier model is instantiated.

-The max_depth argument refers to the size of the decision tree stumps used in gradient boosting.

-The n_estimators argument refers to the number of trees used.

-The n_estimators, max_features, and max_depth parameters are fixed at the defined values. These, like the learning rate, can be optimized, but we'll stick to the default values used in the example above.

A model that performs well on the training set but poorly on the testing set is said to be "overfit." Overfitting is akin to memorizing the answers to an exam: It will help on that particular exam, but not on any others. In other words, overfitting occurs when a model gives undue importance to patterns within a particular dataset that are not found in other, similar datasets. Instead of learning a general pattern that can be applied to other similar datasets, it learns the patterns specific to one dataset.

In [25]:
#using the learning_Rate from the for loop with the best performance we instantate the model and train it

classifier = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)

predictions= classifier.predict(X_test_scaled)


In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.56


In [27]:
cm = confusion_matrix(y_test, predictions)

cm_df=pd.DataFrame(cm, index=["Acutal 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

display(cm_df)

,Predicted 0,Predicted 1
Acutal 0,49,16
Actual 1,39,21


In [28]:
#generate classification report to evaluate the precision

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.56      0.75      0.64        65
           1       0.57      0.35      0.43        60

    accuracy                           0.56       125
   macro avg       0.56      0.55      0.54       125
weighted avg       0.56      0.56      0.54       125

